<h1 align="center">MOM6 boundary fluxes of CFCs and their global budgets</h1> 

This notebook checks if the fluxes of CFCs crossing the surface of the ocean are consistent with corresponding changes in the ocean interior. This particular analysis is for a MOM6/CESM G-compset.

This notebooks analyses the case where:

* Z_RESCALE_POWER = 0                                       
* L_RESCALE_POWER = 0             
* T_RESCALE_POWER = 0


In [14]:
# %pylab inline
import matplotlib.pyplot as plt
import netCDF4
import numpy 

In [15]:
path    = '/glade/scratch/gmarques/gmom.e22.GJRAv3.TL319_t061_zstar_N65.nuopc.CFC_test.004/run/'
static  = netCDF4.Dataset(path+'ocean.mom6.static.nc')
forcing = netCDF4.Dataset(path+'ocean.mom6.frc_1985.nc')
surface = netCDF4.Dataset(path+'ocean.mom6.sfc_1985.nc') ; tvar = 'time'

cp   = 3992.0
rho0 = 1035.0
n    = 4
print('Variables saved:')
for v in forcing.variables: print (v),
for v in surface.variables: print (v),
for v in static.variables: print (v),

Variables saved:
xh
yh
time
nv
xq
yq
scalar_axis
cfc11_flux
cfc12_flux
taux
tauy
ustar
PRCmE
lprec
fprec
evap
vprec
lrunoff
frunoff
seaice_melt
net_heat_coupler
net_heat_surface
frazil
sensible
latent
LwLatSens
SW
LW
Heat_PmE
seaice_melt_heat
heat_content_lrunoff
heat_content_frunoff
heat_content_icemelt
heat_content_lprec
heat_content_fprec
heat_content_vprec
heat_content_cond
heat_content_massout
heat_content_massin
heat_content_surfwater
internal_heat
heat_added
hfds
p_surf
salt_flux
salt_flux_in
vprec_global_adjustment
net_fresh_water_global_adjustment
salt_flux_global_restoring_adjustment
net_massout
net_massin
average_T1
average_T2
average_DT
time_bnds
xh
yh
time
xq
yq
zl
SSH
SST
SSS
SSU
SSV
mass_wt
temp_int
salt_int
KPP_OBLdepth
CFC_11
CFC_12
xh
yh
time
xq
yq
geolon
geolat
geolon_c
geolat_c
geolon_u
geolat_u
geolon_v
geolat_v
area_t
depth_ocean
wet
wet_c
wet_u
wet_v
Coriolis


In [19]:
# This section fills the fields used in this notebook.

#--------------------------------------------------------------
# geometric factors 
lon  = static.variables['geolon'][:]
lat  = static.variables['geolat'][:]
wet  = static.variables['wet'][:]
area = static.variables['area_t'][:]*wet


#--------------------------------------------------------------
# time in days 
time = surface.variables[tvar][:]*86400.


#--------------------------------------------------------------
# cfcs fluxes 
cfc11_flux = forcing.variables['cfc11_flux'][n-1]
cfc12_flux = forcing.variables['cfc12_flux'][n-1]


#--------------------------------------------------------------
CFC_11 = surface.variables['CFC_11']
CFC_12  = surface.variables['CFC_12']



<hr --- />

<h1 align="center">CFC fluxes and their global budget</h1>

We compute the change in total CFCs over a given time period.  Two different methods are used, and the two methods should agree at the level of truncation error.  Note that "truncation error" precision is somewhat larger using offline diagnostics relative to online calculations, particularly if single precision output is saved rather than double precision.  

The net CFC per time of water (units of kg/s) entering through the ocean boundaries is given by the area integral
$$\begin{equation*}
\mbox{mass of CFC entering the ocean surface} = \int CFC_{flux} \, dA,
\end{equation*}$$

The time change of CFC mass is computed according to 
$$\begin{equation*}
\mbox{CFC mass change} = 
\frac{1}{\tau_{n+1} - \tau_{n} } \int dA \left(\int (CFC_{n+1} - CFC_{n}) \, \mathrm{d}z \right) 
\end{equation*}$$
where $\tau_{n+1} - \tau_{n}$ is the time increment in seconds.  Note that we make use of the logfile created by MOM6 (cesm.log.*) as one way to diagnose the total stocks of CFC's

 
$$\begin{equation*}
\boxed{
 \mbox{mass of CFC entering the ocean surface} = \mbox{global change in CFC mass}.
}
\end{equation*}$$

In [20]:
# from cesm.log.8555099.chadmin1.ib0.cheyenne.ucar.edu.210604-133455.gz
CFC_11_stock = [0., 2.0329170279685589E+06, 3.9211172201609621E+06, 5.6371090582493553E+06, 
                7.2482701922035646E+06]
CFC_12_stock = [0., 1.0074530812176701E+06, 1.9423505227430942E+06, 2.7935483468045667E+06, 
                3.5940815281534065E+06]

In [23]:
n0       = n-1
dmass_cfc1 = CFC_11_stock[n] - CFC_11_stock[n0]
dmass_cfc2 = CFC_12_stock[n] - CFC_12_stock[n0]
dt = time[n] - time[n0]
lhs_cfc11 = dmass_cfc1 
rhs_cfc11 = area * ( cfc11_flux ) * dt

lhs_cfc12 = dmass_cfc2
rhs_cfc12 = area * ( cfc12_flux ) * dt

print ('Total CFC11 mass at time step n  [mol]  =',CFC_11_stock[n])
print ('Total CFC11 mass at time step n0  [mol] =',CFC_11_stock[n0])
print ('Total CFC11 mass content change [mol]   =',lhs_cfc11)
print ('Net CFC11 mass through boundaries [mol] =',rhs_cfc11.sum())
print ('Non-dimensional residual for CFC11 (based on difference)  =',(  lhs_cfc11 - rhs_cfc11.sum() )/lhs_cfc11)

print ('Total CFC12 mass at time step n  [mol]  =',CFC_12_stock[n])
print ('Total CFC12 mass at time step n0  [mol] =',CFC_12_stock[n0])
print ('Total CFC12 mass content change [mol]   =',lhs_cfc12)
print ('Net CFC12 mass through boundaries [mol] =',rhs_cfc12.sum())
print ('Non-dimensional residual for CFC12 (based on difference)  =',(  lhs_cfc12 - rhs_cfc12.sum() )/lhs_cfc12)



#print ('Non-dimensional residual (based on difference)     =',(  lhs.sum() - rhs.sum() )/lhs.sum())
#print ('Non-dimensional residual (based on mass_wt[n])     =',(  lhs.sum() - rhs.sum() )/(mass_wt[n]*area).sum())

Total CFC11 mass at time step n  [mol]  = 7248270.192203565
Total CFC11 mass at time step n0  [mol] = 5637109.058249355
Total CFC11 mass content change [mol]   = 1611161.1339542093
Net CFC11 mass through boundaries [mol] = 1611161.133954221
Non-dimensional residual for CFC11 (based on difference)  = -7.22555425236837e-15
Total CFC12 mass at time step n  [mol]  = 3594081.5281534065
Total CFC12 mass at time step n0  [mol] = 2793548.3468045667
Total CFC12 mass content change [mol]   = 800533.1813488398
Net CFC12 mass through boundaries [mol] = 800533.1813488443
Non-dimensional residual for CFC12 (based on difference)  = -5.671467038506209e-15
